In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import h5py
import pandas as pd

In [3]:
dataset_path = 'C:/BBBC021/BBBC021_cleaned_preprocessed.h5'
metadata_path = '../data/raw/CPfeatures.csv'
output_path = '../data/raw/metadata_dataset.h5'

# open metadata file with pandas
cp_features = pd.read_csv(metadata_path, index_col="single_img_name")

In [4]:
cp_features.head()

,Unnamed: 0,ImageNumber,ObjectNumber,Metadata_folder,Metadata_well,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,...,Nuclei_Texture_Variance_CorrTub_3_00,dapi_name,actin_name,tubulin_name,plate_well,Image_Metadata_Plate_DAPI,Image_Metadata_Well_DAPI,Image_Metadata_Compound,Image_Metadata_Concentration,moa
single_img_name,,,,,,,,,,,,,,,,,,,,,
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_0.npy,0,1.0,7.0,Week10_40111,B02,0.193999,-2.057102,-0.689976,-0.104216,0.646993,...,-0.153000,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_1.npy,1,1.0,8.0,Week10_40111,B02,-0.562693,1.533842,-0.876157,-0.486278,-0.768380,...,0.538601,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_2.npy,2,1.0,9.0,Week10_40111,B02,0.717717,-1.877555,-0.317613,-0.660598,-1.629161,...,1.648684,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_3.npy,3,1.0,10.0,Week10_40111,B02,0.017374,-0.441177,0.427112,-0.475127,-1.047884,...,-0.149809,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_4.npy,4,1.0,11.0,Week10_40111,B02,-0.226017,-1.698008,-2.737971,0.912409,1.299734,...,-0.165810,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO


In [5]:
# remove all integer columns
metadata = cp_features.select_dtypes(exclude=['int64', 'float64'])
# add back "Image_Metadata_Concentration" column
metadata.insert(metadata.columns.get_loc('moa'), 'Image_Metadata_Concentration', cp_features['Image_Metadata_Concentration'])
metadata.head()

,Metadata_folder,Metadata_well,dapi_name,actin_name,tubulin_name,plate_well,Image_Metadata_Plate_DAPI,Image_Metadata_Well_DAPI,Image_Metadata_Compound,Image_Metadata_Concentration,moa
single_img_name,,,,,,,,,,,
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_0.npy,Week10_40111,B02,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_1.npy,Week10_40111,B02,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_2.npy,Week10_40111,B02,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_3.npy,Week10_40111,B02,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_4.npy,Week10_40111,B02,Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3...,Week10_200907_B02_s1_w2D492FCE4-15C2-4C66-99A5...,Week10_200907_B02_s1_w436D0A3BC-098D-4271-B5AA...,Week10_40111_B02,Week10_40111,B02,DMSO,0.0,DMSO


In [6]:
# remove ["Unnamed: 0", "Metadata_folder", "dapi_name", "actin_name", "tubulin_name", "Image_Metadata_Well_DAPI"]
metadata_cleaned = metadata.drop(columns=["Metadata_folder", "Metadata_well", "dapi_name", "actin_name", "tubulin_name", "Image_Metadata_Well_DAPI", "Image_Metadata_Plate_DAPI"])
# rename ["plate_well", "Image_Metadata_Compound", "Image_Metadata_Concentration"] to ["well", "compound", "concentration"]
metadata_cleaned = metadata_cleaned.rename(columns={
    "plate_well": "well",
    "Image_Metadata_Compound": "compound",
    "Image_Metadata_Concentration": "concentration"
})
metadata_cleaned.head()

,well,compound,concentration,moa
single_img_name,,,,
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_0.npy,Week10_40111_B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_1.npy,Week10_40111_B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_2.npy,Week10_40111_B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_3.npy,Week10_40111_B02,DMSO,0.0,DMSO
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_4.npy,Week10_40111_B02,DMSO,0.0,DMSO


In [7]:
# List length of unique wells, compounds, concentrations and MOAs
unique_wells = metadata_cleaned['well'].nunique()
unique_compounds = metadata_cleaned['compound'].nunique()
unique_concentrations = metadata_cleaned['concentration'].nunique()
unique_moas = metadata_cleaned['moa'].nunique()
print(f"Unique wells: {unique_wells}, Unique compounds: {unique_compounds}, Unique concentrations: {unique_concentrations}, Unique MOAs: {unique_moas}")

Unique wells: 962, Unique compounds: 39, Unique concentrations: 19, Unique MOAs: 13


In [35]:
# List number of unique compond and concentration pairs
compound_concentration_pairs = metadata_cleaned.groupby(['compound', 'concentration']).size().reset_index(name='count')
print(f"Unique compound-concentration pairs: {len(compound_concentration_pairs)}")

Unique compound-concentration pairs: 104


In [36]:
# List number of unique compond and MOA pairs
compound_moa_pairs = metadata_cleaned.groupby(['compound', 'moa']).size().reset_index(name='count')
print(f"Unique compound-MOA pairs: {len(compound_moa_pairs)}")

Unique compound-MOA pairs: 39


In [70]:
# check that all wells do not have more than one compound
wells_with_multiple_compounds = metadata_cleaned.groupby('well')['compound'].nunique()
wells_with_multiple_compounds = wells_with_multiple_compounds[wells_with_multiple_compounds > 1]
if not wells_with_multiple_compounds.empty:
    print("Wells with multiple compounds:")
    print(wells_with_multiple_compounds)
else:
    print("No wells with multiple compounds found.")

No wells with multiple compounds found.


In [ ]:
# Create HDF5 dataset with string columns for metadata
with h5py.File(dataset_path, 'r') as in_f, h5py.File(output_path, 'a') as out_f:
    raw_names = in_f['image_names']
    total = len(raw_names)

    # Define variable-length UTF-8 string dtype
    str_dtype = h5py.string_dtype(encoding='utf-8')

    # Preallocate datasets with full length
    well_ds = out_f.create_dataset('metadata_well', shape=(total,), dtype=str_dtype, compression='gzip')
    compound_ds = out_f.create_dataset('metadata_compound', shape=(total,), dtype=str_dtype, compression='gzip')
    concentration_ds = out_f.create_dataset('metadata_concentration', shape=(total,), dtype=str_dtype, compression='gzip')
    moa_ds = out_f.create_dataset('metadata_moa', shape=(total,), dtype=str_dtype, compression='gzip')

    for i in tqdm(range(total), desc="Processing Metadata"):
        name = raw_names[i].decode('utf-8')

        if name not in metadata_cleaned.index:
            raise KeyError(f"Image name {name} not found in metadata index.")

        row = metadata_cleaned.loc[name]

        # Directly write to preallocated datasets
        well_ds[i] = row['well']
        compound_ds[i] = row['compound']
        concentration_ds[i] = str(row['concentration'])  # store as string
        moa_ds[i] = row['moa']

print(f"\n✅ Saved metadata for {total} images to: {output_path}")

Processing Metadata: 100%|██████████| 480763/480763 [05:28<00:00, 1461.94it/s]


✅ Saved metadata for 480763 images to: ../data/raw/metadata_dataset.h5


In [52]:
# load the first 10 rows of metadata
with h5py.File(output_path, 'r') as f:
    for key in f.keys():
        print(f"{key}: {f[key][:5]}")  # Print first 5 entries of each dataset

metadata_compound: [b'DMSO' b'DMSO' b'DMSO' b'DMSO' b'DMSO']
metadata_concentration: [b'0.0' b'0.0' b'0.0' b'0.0' b'0.0']
metadata_moa: [b'DMSO' b'DMSO' b'DMSO' b'DMSO' b'DMSO']
metadata_well: [b'Week4_27521_B02' b'Week4_27521_B02' b'Week4_27521_B02'
 b'Week4_27521_B02' b'Week4_27521_B02']


In [54]:
# print length of each dataset
with h5py.File(output_path, 'r') as f:
    for key in f.keys():
        print(f"{key} length: {len(f[key])}")  # Print length of each dataset

# print length of image dataset
with h5py.File(dataset_path, 'r') as f:
    print(f"Image dataset length: {len(f['image_names'])}")  # Print length of image dataset

metadata_compound length: 480763
metadata_concentration length: 480763
metadata_moa length: 480763
metadata_well length: 480763
Image dataset length: 480763


In [64]:
# give me all info on image nr 2
i = 200000
with h5py.File(output_path, 'r') as f:
    print(f"Metadata well: {f['metadata_well'][i]}")
    print(f"Metadata compound: {f['metadata_compound'][i]}")
    print(f"Metadata concentration: {f['metadata_concentration'][i]}")
    print(f"Metadata moa: {f['metadata_moa'][i]}")

with h5py.File(dataset_path, 'r') as f:
    print(f"Image name: {f['image_names'][i].decode('utf-8')}")
    print(f"Image shape: {f['images'][i].shape}")  # Assuming 'images' is the dataset containing image data
    print(f"Image dtype: {f['images'][i].dtype}")  # Assuming 'images' is the dataset containing image data

Metadata well: b'Week3_25701_B04'
Metadata compound: b'demecolcine'
Metadata concentration: b'3.0'
Metadata moa: b'Microtubule destabilizers'
Image name: Week3_290607_B04_s1_w13D9CD430-DDC4-47DC-BC8B-FDA9447A5594_37.npy
Image shape: (3, 64, 64)
Image dtype: float32


In [20]:
# List all the different concentrations in the dataset
with h5py.File(output_path, 'r') as f:
    unique_concentrations = set(f['metadata_concentration'][:])
    print(f"Unique concentrations in the dataset: {unique_concentrations}")
    print(f"Number of unique concentrations: {len(unique_concentrations)}")

Unique concentrations in the dataset: {b'0.003', b'20.0', b'6.0', b'1.5', b'30.0', b'15.0', b'0.01', b'2.0', b'100.0', b'0.001', b'1.0', b'0.0', b'0.03', b'10.0', b'0.3', b'50.0', b'3.0', b'0.1', b'5.0'}
Number of unique concentrations: 19


# CP correlation

In [23]:
# Get all int and float columns from the original metadata
cellprofiler_features = cp_features.select_dtypes(include=['int64', 'float64'])
cellprofiler_features.drop(columns=['Unnamed: 0', 'ImageNumber', 'Image_Metadata_Concentration','ObjectNumber'], inplace=True)  # Drop the index column if it exists
cellprofiler_features

,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MaxFeretDiameter,Cytoplasm_AreaShape_MaximumRadius,...,Nuclei_Texture_SumVariance_CorrDAPI_10_00,Nuclei_Texture_SumVariance_CorrDAPI_3_00,Nuclei_Texture_SumVariance_CorrTub_10_00,Nuclei_Texture_SumVariance_CorrTub_3_00,Nuclei_Texture_Variance_CorrActin_10_00,Nuclei_Texture_Variance_CorrActin_3_00,Nuclei_Texture_Variance_CorrDAPI_10_00,Nuclei_Texture_Variance_CorrDAPI_3_00,Nuclei_Texture_Variance_CorrTub_10_00,Nuclei_Texture_Variance_CorrTub_3_00
single_img_name,,,,,,,,,,,,,,,,,,,,,
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_0.npy,0.193999,-2.057102,-0.689976,-0.104216,0.646993,1.125279,-0.297727,0.508526,0.374853,0.340593,...,-0.066568,-0.083820,-0.158029,-0.147652,-0.195899,-0.204407,-0.076131,-0.096239,-0.138961,-0.153000
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_1.npy,-0.562693,1.533842,-0.876157,-0.486278,-0.768380,0.803726,0.671996,-1.066925,-1.072387,-0.805709,...,-0.285345,-0.269458,0.441284,0.538651,-0.190330,-0.198416,-0.297608,-0.277298,0.606922,0.538601
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_2.npy,0.717717,-1.877555,-0.317613,-0.660598,-1.629161,0.080157,-0.424397,0.406612,0.432122,1.376551,...,0.084520,-0.039017,1.957213,1.724831,-0.186461,-0.193917,-0.055597,-0.065842,1.550169,1.648684
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_3.npy,0.017374,-0.441177,0.427112,-0.475127,-1.047884,-0.305084,-1.088986,-0.198949,-0.176925,0.506117,...,-0.276352,-0.261815,-0.178935,-0.155573,-0.200005,-0.209290,-0.282177,-0.271378,-0.123574,-0.149809
Week10_200907_B02_s1_w18E215662-2CF7-4739-93F3-DBD0C40B78DB_4.npy,-0.226017,-1.698008,-2.737971,0.912409,1.299734,-1.537763,-0.148227,0.384040,0.185869,-0.654728,...,-0.226992,-0.240634,-0.191919,-0.165251,-0.200507,-0.208415,-0.261484,-0.249309,-0.149291,-0.165810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Week9_090907_G11_s4_w1C447A151-1F85-4E19-9C96-97BF0C969EA4_69.npy,0.624906,-0.979819,0.054749,-0.607595,-0.674608,1.459895,0.744605,0.485588,0.611181,1.561836,...,-0.301246,-0.289768,-0.222936,-0.241942,-0.197183,-0.206319,-0.312452,-0.297580,-0.240166,-0.248631
Week9_090907_G11_s4_w1C447A151-1F85-4E19-9C96-97BF0C969EA4_70.npy,0.005063,0.995200,0.054749,-0.724489,-1.643694,0.647541,0.821980,-0.364929,-0.417532,0.643406,...,-0.295334,-0.292337,-0.211584,-0.228885,-0.200589,-0.209526,-0.318822,-0.299646,-0.229312,-0.233790
Week9_090907_G11_s4_w1C447A151-1F85-4E19-9C96-97BF0C969EA4_71.npy,-0.014352,-0.261630,0.240931,-0.476843,0.128440,1.069168,0.485922,-0.051005,-0.075211,0.298848,...,-0.287428,-0.282814,-0.206272,-0.221808,-0.199981,-0.209036,-0.301267,-0.288597,-0.219446,-0.227377
